Here I prepared the model for racing car. Some image preprocessing, loading the dataset, creating & training model, and exporting it to the file. This code was run on Google Colab.

In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

Here I tried some image preprocessing; as a result only resize is left cause car's CPU is not strong enough to preprocess every image fast enough.

In [2]:
import cv2
import numpy as np

def preprocess_image(image, target_size=(64, 64), reduce_colors=False):
    # resize image
    image_resized = cv2.resize(image, target_size)

    # normalize
    image_normalized = image_resized / 255.0


    # decrease the amount of colors
    if reduce_colors:
        # convert images to rgb for k-means
        image_rgb = cv2.cvtColor((image_normalized * 255).astype(np.uint8), cv2.COLOR_BGR2RGB)

        # k-means
        pixels = image_rgb.reshape((-1, 3))
        k = 8  # amount of colors
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 0.1)
        _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
        centers = np.uint8(centers)
        segmented_image = centers[labels.flatten()]
        image_reduced = segmented_image.reshape(image_rgb.shape)

        # return back to normalized format
        image_reduced_normalized = image_reduced / 255.0

        return image_reduced_normalized

    return image_normalized


Here I load the labels from txt file.

In [3]:
def load_labels(label_file):
    labels = {}
    with open(label_file, 'r') as file:
        for line in file.readlines():
            try:
                key, value = line.strip().split(' == ')
            except Exception as e:
                print(line)
                print(e)
            value = float(value)
            labels[int(key)] = value
    return labels

Loads images and their labels


In [4]:
def load_and_preprocess_images(image_folder, label_file, target_size=(64, 64)):
    # load labels
    labels = load_labels(label_file)

    images = []
    image_labels = []

    # load images
    for filename in os.listdir(image_folder):
        if filename.endswith(".png") or filename.endswith(".jpg"):

            image_id = int(filename.split('.')[0])
            img_path = os.path.join(image_folder, filename)

            if image_id in labels:

                image = cv2.imread(img_path)
                processed_image = preprocess_image(image, target_size)

                images.append(processed_image)
                label = labels[image_id]
                image_labels.append(label)

                # mirror reflection on the turns
                if label >= 0.1 or label <= -0.1:
                    mirrored_image = np.fliplr(processed_image)
                    mirrored_label = -label

                    images.append(mirrored_image)
                    image_labels.append(mirrored_label)

    images = np.array(images)
    image_labels = np.array(image_labels)

    return images, image_labels

In [5]:
# load and preprocess images
image_folder = '/content/drive/MyDrive/image_regressor/images'
X, y = load_and_preprocess_images(image_folder, "/content/drive/MyDrive/image_regressor/images/road_following.txt")



Split the dataset into:
* Train - 60%
* Validation - 20%
* Test - 20%

In [14]:
# split data into train, validation, and test datasets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

Here I create the Convolutional Neural Network(CNN) - not very deep one, however this is enough, and again, the CPU & GPU of the car are not strong enough for fast processing.

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# create model
model = Sequential()

# First convolutional layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))

# Second convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Third convolutional layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Conversion to one-dimensional vector
model.add(Flatten())

# Fully connected layer with dropout for regularisation
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(1, activation='tanh'))  # Значение от -1 до 1

# Model compilation
model.compile(optimizer='adam', loss='mean_squared_error')

# Output the structure of the model
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 12, 12, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │       1,179,904 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,273,409 (4.86 MB)

 Trainable params: 1,273,409 (4.86 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.fit(X_train, y_train, batch_size=32, epochs=35, validation_data=(X_val, y_val))

Epoch 1/35
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 272ms/step - loss: 0.5901 - val_loss: 0.4292
Epoch 2/35
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 226ms/step - loss: 0.4295 - val_loss: 0.4247
Epoch 3/35
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 360ms/step - loss: 0.3987 - val_loss: 0.4003
Epoch 4/35
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 238ms/step - loss: 0.3537 - val_loss: 0.4066
Epoch 5/35
25/25 ━━━━━━━━━━━━━━━━━━━━ 12s 304ms/step - loss: 0.3354 - val_loss: 0.2523
Epoch 6/35
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 299ms/step - loss: 0.1997 - val_loss: 0.1505
Epoch 7/35
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 229ms/step - loss: 0.1304 - val_loss: 0.1130
Epoch 8/35
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 250ms/step - loss: 0.0949 - val_loss: 0.0777
Epoch 9/35
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 236ms/step - loss: 0.0698 - val_loss: 0.0721
Epoch 10/35
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 226ms/step - loss: 0.0635 - val_loss: 0.0571
Epoch 11/35
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 267ms/step - loss: 0.0546 - val_loss: 0.0511
Epoch 12/35
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 220ms

In [20]:
# compare the pred
for i, j in zip(model.predict(X_test)[:15], y_test[:15]):
    print(*i, j, sep='\t')

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step
-0.85190773	-0.8
-0.7797206	-0.8
0.24608402	0.1
0.9238737	1.0
-0.69539076	-0.5
0.96619684	1.0
0.07830587	0.4
-0.9792851	-1.0
0.8410874	1.0
0.66843957	0.5
-0.7087476	-0.4
0.94741064	1.0
0.93123496	1.0
0.98917484	1.0
-0.113700174	-0.2


And then we need to convert our model to ONNX to make it runable on the car(as most of the latest versions of TF are not runable)

In [ ]:
pip install onnx tf2onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 25.8 MB/s eta 0:00:00


In [ ]:
import tf2onnx


# Convert model to ONNX
# The opset is set according to the version that is on the car ONNX Runtime;
# You should look for this value in the documentation for your case
onnx_model, _ = tf2onnx.convert.from_keras(model, opset=13)

# And finally save it
with open('keras_inversed_images_4.onnx', 'wb') as f:
    f.write(onnx_model.SerializeToString())
